In [203]:
import math
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from matplotlib import figure
import seaborn as sn

import os
os.environ['PATH'] = os.environ['PATH']+';'+os.environ['CONDA_PREFIX']+r"\Library\bin\graphviz"
import pydotplus
from sklearn import tree
from IPython.display import Image

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from scipy.stats import mode
from collections import Counter
from scipy import stats
%matplotlib inline

df = pd.read_csv(r'C:\Users\Mario\Desktop\UniPi\1° semestre\Data Mining 1\Datasets\DatasetProject1\df_originale.csv')

df.head()


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,HourlyRate,EnvironmentSatisfaction,Gender,...,PercentSalaryHike,WorkLifeBalance,MonthlyIncome,TrainingTimesLastYear,NumCompaniesWorked,MaritalStatus,PerformanceRating,TotalWorkingYears,YearsAtCompany,YearsInCurrentRole
0,27.0,Yes,1,Sales,15,2,Medical,59,1,1,...,14,2,4298.0,1.0,6,0,3.0,7,2.0,2
1,30.0,Yes,2,Sales,16,4,Marketing,70,1,1,...,13,3,5304.0,2.0,2,1,4.0,10,8.0,0
2,56.0,No,0,Research & Development,2,4,Technical Degree,60,4,1,...,11,2,6306.0,2.0,2,0,4.0,12,12.0,3
3,41.0,Yes,2,Human Resources,18,5,Human Resources,89,4,1,...,17,3,6430.0,4.0,0,1,3.0,2,2.0,0
4,42.0,No,2,Research & Development,1,3,Life Sciences,76,4,1,...,22,3,2766.0,6.0,3,1,4.0,12,5.0,3


In [204]:
df.shape

(1459, 22)

### TRASFORMAZIONE DA VARIABILI NUMERICHE A CATEGORICHE

In [205]:
distance_bins=[-1, 4, 10, 30]
distance = ["0-4 miles", "4-10 miles", "10-30 miles"]
df["DistanceFromHome"] = pd.cut(df["DistanceFromHome"], distance_bins, labels=distance)

age_bins=[17, 32, 39, 60]
age = ["Age:18-32", "Age:32-39", "Age:39-60"]
df["Age"] = pd.cut(df["Age"], age_bins, labels=age)

hourly_bins=[29, 44, 58, 72, 86, 100]
hourly = ["Hourly:30-44", "Hourly:44-58", "Hourly:58-72", "Hourly:72-86", "Hourly:86-100"]
df["HourlyRate"] = pd.cut(df["HourlyRate"], hourly_bins, labels=hourly)

percent_bins=[10, 18, 25]
percent = ["Hike:11-18(%)", "Hike:18-25(%)"]
df["PercentSalaryHike"] = pd.cut(df["PercentSalaryHike"], percent_bins, labels=percent)

training_bins=[-1, 2, 4, 6]
training = ["Training:0-2", "Training:2-4", "Training:4-6"]
df["TrainingTimesLastYear"] = pd.cut(df["TrainingTimesLastYear"], training_bins, labels=training)

companies_bins=[-1, 3, 6, 9]
companies = ["Companies:0-3", "Companies:3-6", "Companies:6-9"]
df["NumCompaniesWorked"] = pd.cut(df["NumCompaniesWorked"], companies_bins, labels=companies)

total_bins=[0, 5, 15, 40]
total = ["TotWork:1-5", "TotWork:5-15", "TotWork:15-40"]
df["TotalWorkingYears"] = pd.cut(df["TotalWorkingYears"], total_bins, labels=total)

atcompany_bins=[-1, 2, 6, 29]
atcompany = ["AtCompany:0-2", "AtCompany:2-6", "AtCompany:6-29"]
df["YearsAtCompany"] = pd.cut(df["YearsAtCompany"], atcompany_bins, labels=atcompany)

current_bins=[-1, 6, 12, 18]
current = ["CurrRole:0-6", "CurrRole:6-12", "CurrRole:12-18"]
df["YearsInCurrentRole"] = pd.cut(df["YearsInCurrentRole"], current_bins, labels=current)

monthly_bins=[1000, 3000, 5000, 8000, 20000]
monthly = ["Monthly:1000-3000", "Monthly:3000-5000", "Monthly:5000-8000", "Monthly:8000-20000"]
df["MonthlyIncome"] = pd.cut(df["MonthlyIncome"], monthly_bins, labels=monthly)

In [206]:
for i in range(0, len(df.Gender)):
    if df.Gender.iloc[i] == 1:
        df.Gender.iloc[i] = "Male"
    else:
        df.Gender.iloc[i] = "Female"
        
for i in range(0, len(df.BusinessTravel)):
    if df.BusinessTravel.iloc[i] == 0:
        df.BusinessTravel.iloc[i] = "Non_travel"
    elif df.BusinessTravel.iloc[i] == 1:
        df.BusinessTravel.iloc[i] = "Travel_frequently"
    elif df.BusinessTravel.iloc[i] == 2:
        df.BusinessTravel.iloc[i] = "Travel_rarely"
        
for i in range(0, len(df.JobSatisfaction)):
    if df.JobSatisfaction.iloc[i] == 1:
        df.JobSatisfaction.iloc[i] = "JobSatisf:Low"
    elif df.JobSatisfaction.iloc[i] == 2:
        df.JobSatisfaction.iloc[i] = "JobSatisf:Medium"
    elif df.JobSatisfaction.iloc[i] == 3:
        df.JobSatisfaction.iloc[i] = "JobSatisf:High"
    elif df.JobSatisfaction.iloc[i] == 4:
        df.JobSatisfaction.iloc[i] = "JobSatisf:Very High"
        
for i in range(0, len(df.EnvironmentSatisfaction)):
    if df.EnvironmentSatisfaction.iloc[i] == 1:
        df.EnvironmentSatisfaction.iloc[i] = "EnvSatisf:Low"
    elif df.EnvironmentSatisfaction.iloc[i] == 2:
        df.EnvironmentSatisfaction.iloc[i] = "EnvSatisf:Medium"
    elif df.EnvironmentSatisfaction.iloc[i] == 3:
        df.EnvironmentSatisfaction.iloc[i] = "EnvSatisf:High"
    elif df.EnvironmentSatisfaction.iloc[i] == 4:
        df.EnvironmentSatisfaction.iloc[i] = "EnvSatisf:Very High"
        
for i in range(0, len(df.Education)):
    if df.Education.iloc[i] == 1:
        df.Education.iloc[i] = "Below College"
    elif df.Education.iloc[i] == 2:
        df.Education.iloc[i] = "College"
    elif df.Education.iloc[i] == 3:
        df.Education.iloc[i] = "Bachelor"
    elif df.Education.iloc[i] == 4:
        df.Education.iloc[i] = "Master"
    elif df.Education.iloc[i] == 5:
        df.Education.iloc[i] = "Doctor"
        
for i in range(0, len(df.PerformanceRating)):
    if df.PerformanceRating.iloc[i] == 1:
        df.PerformanceRating.iloc[i] = "Performance:Low"
    elif df.PerformanceRating.iloc[i] == 2:
        df.PerformanceRating.iloc[i] = "Performance:Good"
    elif df.PerformanceRating.iloc[i] == 3:
        df.PerformanceRating.iloc[i] = "Performance:Excellent"
    elif df.PerformanceRating.iloc[i] == 4:
        df.PerformanceRating.iloc[i] = "Performance:Outstanding"
        
for i in range(0, len(df.WorkLifeBalance)):
    if df.WorkLifeBalance.iloc[i] == 1:
        df.WorkLifeBalance.iloc[i] = "WLBalance:Bad"
    elif df.WorkLifeBalance.iloc[i] == 2:
        df.WorkLifeBalance.iloc[i] = "WLBalance:Good"
    elif df.WorkLifeBalance.iloc[i] == 3:
        df.WorkLifeBalance.iloc[i] = "WLBalance:Better"
    elif df.WorkLifeBalance.iloc[i] == 4:
        df.WorkLifeBalance.iloc[i] = "WLBalance:Best"
        
for i in range(0, len(df.MaritalStatus)):
    if df.MaritalStatus.iloc[i] == 0:
        df.MaritalStatus.iloc[i] = "Divorced"
    elif df.MaritalStatus.iloc[i] == 1:
        df.MaritalStatus.iloc[i] = "Married"
    elif df.MaritalStatus.iloc[i] == 2:
        df.MaritalStatus.iloc[i] = "Single"

for i in range(0, len(df.Attrition)):
    if df.Attrition.iloc[i] == "Yes":
        df.Attrition.iloc[i] = "Attr:Yes"
    elif df.Attrition.iloc[i] == "No":
        df.Attrition.iloc[i] = "Attr:No"
        
for i in range(0, len(df.OverTime)):
    if df.OverTime.iloc[i] == "Yes":
        df.OverTime.iloc[i] = "OverTime:Yes"
    elif df.OverTime.iloc[i] == "No":
        df.OverTime.iloc[i] = "OverTime:No"


df.head()

C:\Users\Mario\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,HourlyRate,EnvironmentSatisfaction,Gender,...,PercentSalaryHike,WorkLifeBalance,MonthlyIncome,TrainingTimesLastYear,NumCompaniesWorked,MaritalStatus,PerformanceRating,TotalWorkingYears,YearsAtCompany,YearsInCurrentRole
0,Age:18-32,Attr:Yes,Travel_frequently,Sales,10-30 miles,College,Medical,Hourly:58-72,EnvSatisf:Low,Male,...,Hike:11-18(%),WLBalance:Good,Monthly:3000-5000,Training:0-2,Companies:3-6,Divorced,Performance:Excellent,TotWork:5-15,AtCompany:0-2,CurrRole:0-6
1,Age:18-32,Attr:Yes,Travel_rarely,Sales,10-30 miles,Master,Marketing,Hourly:58-72,EnvSatisf:Low,Male,...,Hike:11-18(%),WLBalance:Better,Monthly:5000-8000,Training:0-2,Companies:0-3,Married,Performance:Outstanding,TotWork:5-15,AtCompany:6-29,CurrRole:0-6
2,Age:39-60,Attr:No,Non_travel,Research & Development,0-4 miles,Master,Technical Degree,Hourly:58-72,EnvSatisf:Very High,Male,...,Hike:11-18(%),WLBalance:Good,Monthly:5000-8000,Training:0-2,Companies:0-3,Divorced,Performance:Outstanding,TotWork:5-15,AtCompany:6-29,CurrRole:0-6
3,Age:39-60,Attr:Yes,Travel_rarely,Human Resources,10-30 miles,Doctor,Human Resources,Hourly:86-100,EnvSatisf:Very High,Male,...,Hike:11-18(%),WLBalance:Better,Monthly:5000-8000,Training:2-4,Companies:0-3,Married,Performance:Excellent,TotWork:1-5,AtCompany:0-2,CurrRole:0-6
4,Age:39-60,Attr:No,Travel_rarely,Research & Development,0-4 miles,Bachelor,Life Sciences,Hourly:72-86,EnvSatisf:Very High,Male,...,Hike:18-25(%),WLBalance:Better,Monthly:1000-3000,Training:4-6,Companies:0-3,Married,Performance:Outstanding,TotWork:5-15,AtCompany:2-6,CurrRole:0-6


In [207]:
df.Gender.value_counts()

Male      909
Female    550
Name: Gender, dtype: int64

In [208]:
df.OverTime.value_counts()

OverTime:No     1046
OverTime:Yes     413
Name: OverTime, dtype: int64

In [209]:
df.EnvironmentSatisfaction.value_counts()

EnvSatisf:High         450
EnvSatisf:Very High    442
EnvSatisf:Low          284
EnvSatisf:Medium       283
Name: EnvironmentSatisfaction, dtype: int64

In [210]:
df.Education.value_counts()

Bachelor         566
Master           398
College          279
Below College    168
Doctor            48
Name: Education, dtype: int64

In [211]:
df.PerformanceRating.value_counts()

Performance:Excellent      1260
Performance:Outstanding     199
Name: PerformanceRating, dtype: int64

In [212]:
df.WorkLifeBalance.value_counts()

WLBalance:Better    887
WLBalance:Good      342
WLBalance:Best      151
WLBalance:Bad        79
Name: WorkLifeBalance, dtype: int64

In [213]:
df.MaritalStatus.value_counts()

Married     672
Single      460
Divorced    327
Name: MaritalStatus, dtype: int64

In [214]:
df.Age.value_counts()

Age:32-39    569
Age:39-60    458
Age:18-32    432
Name: Age, dtype: int64

In [215]:
df.HourlyRate.value_counts()

Hourly:72-86     308
Hourly:86-100    298
Hourly:44-58     298
Hourly:30-44     281
Hourly:58-72     274
Name: HourlyRate, dtype: int64

In [216]:
df.PercentSalaryHike.value_counts()

Hike:11-18(%)    1158
Hike:18-25(%)     301
Name: PercentSalaryHike, dtype: int64

In [217]:
df.TrainingTimesLastYear.value_counts()

Training:2-4    774
Training:0-2    531
Training:4-6    154
Name: TrainingTimesLastYear, dtype: int64

In [218]:
df.NumCompaniesWorked.value_counts()

Companies:0-3    1012
Companies:3-6     272
Companies:6-9     175
Name: NumCompaniesWorked, dtype: int64

In [219]:
df.TotalWorkingYears.value_counts()

TotWork:5-15     798
TotWork:15-40    356
TotWork:1-5      305
Name: TotalWorkingYears, dtype: int64

In [220]:
df.YearsAtCompany.value_counts()

AtCompany:2-6     641
AtCompany:0-2     412
AtCompany:6-29    406
Name: YearsAtCompany, dtype: int64

In [221]:
df.YearsInCurrentRole.value_counts()

CurrRole:0-6      974
CurrRole:6-12     439
CurrRole:12-18     46
Name: YearsInCurrentRole, dtype: int64

In [222]:
df.MonthlyIncome.value_counts()

Monthly:3000-5000     471
Monthly:5000-8000     375
Monthly:8000-20000    309
Monthly:1000-3000     304
Name: MonthlyIncome, dtype: int64

In [223]:
df.DistanceFromHome.value_counts()

0-4 miles      563
4-10 miles     455
10-30 miles    441
Name: DistanceFromHome, dtype: int64

In [224]:
df.to_csv(r'C:\Users\Mario\Desktop\UniPi\1° semestre\Data Mining 1\Datasets\DatasetProject1\dfpattern.csv', index = False)